In [6]:
import polars as pl
import duckdb
from pathlib import Path

# Handle paths

In [27]:
staging_tool_path = Path().absolute().parent.parent / "data" / "datalake" / "staging" / "vr_data"

vr_full_load_jsons = list(staging_tool_path.glob("**/*.json"))
print(len(list(vr_full_load_jsons)))

1


# Visualization on one loaded json file

In [28]:
polars_df = pl.read_json(vr_full_load_jsons[0])
polars_df.head(2)


trainNumber,departureDate,operatorUICCode,operatorShortCode,trainType,trainCategory,commuterLineID,runningCurrently,cancelled,version,timetableType,timetableAcceptanceDate,timeTableRows
i64,str,i64,str,str,str,str,bool,bool,i64,str,str,list[struct[16]]
1,"""2023-11-01""",10,"""vr""","""IC""","""Long-distance""","""""",false,false,286822491217,"""REGULAR""","""2023-07-05T11:…","[{""HKI"",1,""FI"",""DEPARTURE"",true,true,""10"",false,""2023-11-01T04:55:00.000Z"",""2023-11-01T04:55:21.000Z"",0,[],1,null,null,{""KUPLA"",true,""2023-11-01T04:53:40.000Z""}}, {""PSL"",10,""FI"",""ARRIVAL"",true,true,""4"",false,""2023-11-01T05:00:00.000Z"",""2023-11-01T05:00:14.000Z"",0,[],1,null,null,{null,null,null}}, … {""JNS"",460,""FI"",""ARRIVAL"",true,true,""1"",false,""2023-11-01T09:41:00.000Z"",""2023-11-01T09:39:41.000Z"",-1,[],1,""2023-11-01T09:39:41.000Z"",""LIIKE_AUTOMATIC"",{null,null,null}}]"
2,"""2023-11-01""",10,"""vr""","""S""","""Long-distance""","""""",false,false,286821604082,"""REGULAR""","""2023-07-05T11:…","[{""JNS"",460,""FI"",""DEPARTURE"",true,true,""1"",false,""2023-11-01T02:58:00.000Z"",""2023-11-01T02:58:00.000Z"",0,[],2,null,null,{""KUPLA"",true,""2023-11-01T02:53:10.000Z""}}, {""PLT"",1070,""FI"",""ARRIVAL"",false,null,"""",false,""2023-11-01T02:58:42.000Z"",null,0,[],2,""2023-11-01T02:58:42.000Z"",""LIIKE_AUTOMATIC"",{null,null,null}}, … {""HKI"",1,""FI"",""ARRIVAL"",true,true,""9"",false,""2023-11-01T07:30:00.000Z"",""2023-11-01T07:31:07.000Z"",1,[],2,null,null,{null,null,null}}]"


In [30]:
SCHEMA = {
    "trainNumber": pl.Int64,
    "departureDate": pl.Utf8,
    "operatorUICCode": pl.Int64,
    "operatorShortCode": pl.Utf8,
    "trainType": pl.Utf8,
	"trainCategory": pl.Utf8,
	"commuterLineID": pl.Utf8,
    "runningCurrently": pl.Boolean,
	"cancelled": pl.Boolean,
    "deleted": pl.Boolean,
    "version": pl.Int64,
    "timetableType": pl.Utf8,
	"timetableAcceptanceDate": pl.Utf8,
    "timetableRows": pl.List(
        pl.Struct({
            "stationShortCode": pl.Utf8,
            "stationUICCode": pl.Int64,
            "countryCode": pl.Utf8,
            "type": pl.Utf8,
            "trainStopping": pl.Boolean,
            "commercialStop": pl.Boolean,
            "commercialTrack": pl.Utf8,
            "cancelled": pl.Boolean,
            "scheduledTime": pl.Utf8,
            "liveEstimateTime": pl.Utf8,
            "estimateSource": pl.Utf8,
            "unknownDelay": pl.Boolean,
            "actualTime": pl.Utf8,
            "differenceInMinutes": pl.Int64,
            "causes": pl.List(),
            "trainReady": pl.List(
                pl.Struct(
                    {
                        "source": pl.Utf8,
                        "accepted": pl.Boolean,
                        "timestamp": pl.Utf8,   
                    }
                ),
            ),
        }),
    )
}

polars_df = pl.read_json(vr_full_load_jsons[0], schema=SCHEMA)
polars_df

trainNumber,departureDate,operatorUICCode,operatorShortCode,trainType,trainCategory,commuterLineID,runningCurrently,cancelled,deleted,version,timetableType,timetableAcceptanceDate,timetableRows
i64,str,i64,str,str,str,str,bool,bool,bool,i64,str,str,list[struct[16]]
1,"""2023-11-01""",10,"""vr""","""IC""","""Long-distance""","""""",false,false,null,286822491217,"""REGULAR""","""2023-07-05T11:…",null
2,"""2023-11-01""",10,"""vr""","""S""","""Long-distance""","""""",false,false,null,286821604082,"""REGULAR""","""2023-07-05T11:…",null
3,"""2023-11-01""",10,"""vr""","""IC""","""Long-distance""","""""",false,false,null,286823761991,"""REGULAR""","""2023-07-05T11:…",null
4,"""2023-11-01""",10,"""vr""","""IC""","""Long-distance""","""""",false,false,null,286822663552,"""REGULAR""","""2023-07-05T11:…",null
5,"""2023-11-01""",10,"""vr""","""IC""","""Long-distance""","""""",false,false,null,286824982323,"""REGULAR""","""2023-07-05T11:…",null
6,"""2023-11-01""",10,"""vr""","""IC""","""Long-distance""","""""",false,false,null,286823288189,"""REGULAR""","""2023-07-05T11:…",null
7,"""2023-11-01""",10,"""vr""","""S""","""Long-distance""","""""",false,false,null,286825745031,"""REGULAR""","""2023-07-05T11:…",null
8,"""2023-11-01""",10,"""vr""","""IC""","""Long-distance""","""""",false,false,null,286824613102,"""REGULAR""","""2023-07-05T11:…",null
9,"""2023-11-01""",10,"""vr""","""IC""","""Long-distance""","""""",false,false,null,286826214328,"""REGULAR""","""2023-07-05T11:…",null


In [31]:
polars_df.head()

trainNumber,departureDate,operatorUICCode,operatorShortCode,trainType,trainCategory,commuterLineID,runningCurrently,cancelled,deleted,version,timetableType,timetableAcceptanceDate,timetableRows
i64,str,i64,str,str,str,str,bool,bool,bool,i64,str,str,list[struct[16]]
1,"""2023-11-01""",10,"""vr""","""IC""","""Long-distance""","""""",false,false,null,286822491217,"""REGULAR""","""2023-07-05T11:…",null
2,"""2023-11-01""",10,"""vr""","""S""","""Long-distance""","""""",false,false,null,286821604082,"""REGULAR""","""2023-07-05T11:…",null
3,"""2023-11-01""",10,"""vr""","""IC""","""Long-distance""","""""",false,false,null,286823761991,"""REGULAR""","""2023-07-05T11:…",null
4,"""2023-11-01""",10,"""vr""","""IC""","""Long-distance""","""""",false,false,null,286822663552,"""REGULAR""","""2023-07-05T11:…",null
5,"""2023-11-01""",10,"""vr""","""IC""","""Long-distance""","""""",false,false,null,286824982323,"""REGULAR""","""2023-07-05T11:…",null
